# **Proyecto Final BD_NR**

> **¿Qué hace este programa?** Realiza una descarga de la API del Art Institute of Chicago de hasta 1000 obras para descargarla a MongoDB, y luego, usando una máquina virtual, prender una instancia y pasar los datos a Neo4j.

**INSTRUCCIONES** Corre la celda a continuación, llena los inputs deseados, y deja correr el resto de las celdas

## **1. Lectura de inputs**

In [ ]:
from IPython.display import display
import ipywidgets as widgets

name_bd_mongo = widgets.Text(value='', description='BD:', layout=widgets.Layout(width='300px'))
name_col_mongo = widgets.Text(value='', description='Colección:', layout=widgets.Layout(width='300px'))
submit_button = widgets.Button(description='Submit')

def submit_button_clicked(b):
    global bd_mongo, col_mongo
    bd_mongo = name_bd_mongo.value
    col_mongo = name_col_mongo.value
    print("BD de Mongo:", bd_mongo)
    print("Colección de Mongo:", col_mongo)

submit_button.on_click(submit_button_clicked)

input_widgets = widgets.VBox([name_bd_mongo, name_col_mongo, submit_button])
display(input_widgets)


## **2. Import de bibliotecas**

In [2]:
import requests
import pymongo
from neo4j import GraphDatabase

## **3. Conexión API a MongoDB**

Conexión a la API del Art Institute de Chicago para descargar 1000 obras de arte en una base de MongoDB.

**Descarga de obras de la API**

In [ ]:
# Crear un arreglo para guardar las obras de arte

url = "https://api.artic.edu/api/v1/artworks?limit=100" # Descargar 500 obras
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print(f'Error en la descarga {response.status_code}')
    
# Descargar solo los datos
collection = data['data']

In [ ]:
collection[1]

**Revisar la estructura de cada JSON de una obra**

**Subir las obras a una nueva BD y colección en MongoDB**

In [4]:
# Crear un cliente de MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')

# Poner nombre a la BD
db = client[bd_mongo]

# Poner nombre a la conexión
my_collection = db[col_mongo]
insert_result = my_collection.insert_many(collection)

NameError: name 'bd_mongo' is not defined

## **4. Conexión de MongoDB a Neo4j**

Conexión de los datos de MongoDB a Neo4j, para así poder crear una nueva base de grafos, relacionando obras de arte entre ellas.

In [3]:
# Conectarse a Neo4j

neo_driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'Putopacman14'))

In [5]:
# Crear un cliente de MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')

# Poner nombre a la BD
db = client["museum"]

# Poner nombre a la conexión
my_collection = db["artwork"]

### **4.1. Conseguir entidades**

In [6]:
# Artistas
pipeline_artists = [
    {"$group": {"_id": "$artist_title"}},
    {"$project": {"_id": 0, "artista": "$_id"}},
    {"$sort": {"artista": 1}}
]

artistas = my_collection.aggregate(pipeline_artists)

# Obras

pipeline_obras = [{ "$project": {
      "nombre": "$title",
      "id": "$id",
      "lugar_origen": "$place_of_origin",
      "artista": "$artist_title",
      "estilo": "$style_title",
      "tipo": "$medium_display",
      "_id": 0
    }
  }
]


obras = my_collection.aggregate(pipeline_obras)


# Lugares

pipeline_lugares = [
  {"$group": {"_id": "$place_of_origin"}},
  {"$project":{"_id":0, "lugar":"$_id"}},
  {"$sort":{"lugar":1}}
]


lugares = my_collection.aggregate(pipeline_lugares)


# Estilo

pipeline_estilos = [
  {"$group": {"_id": "$style_title"}},
  {"$project":{"_id":0, "estilo":"$_id"}},
  {"$sort":{"estilo":1}}
]


estilos = my_collection.aggregate(pipeline_estilos)
